In [1]:
!pip install transformers
!pip install sentence-transformers
!pip install faiss-cpu
!pip install datasets
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.2/494.2 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 106.6 MB/s eta 0:00:00


In [5]:
!pip install datase

In [9]:
!git clone https://github.com/czyssrs/FinQA.git

Cloning into 'FinQA'...
remote: Enumerating objects: 239, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 239 (delta 26), reused 23 (delta 23), pack-reused 181 (from 1)
Receiving objects: 100% (239/239), 26.78 MiB | 24.93 MiB/s, done.
Resolving deltas: 100% (118/118), done.


In [10]:
import json

with open("FinQA/dataset/train.json", "r") as f:
    train_data = json.load(f)

print("Total samples:", len(train_data))
print(train_data[0].keys())

Total samples: 6251
dict_keys(['pre_text', 'post_text', 'filename', 'table_ori', 'table', 'qa', 'id', 'table_retrieved', 'text_retrieved', 'table_retrieved_all', 'text_retrieved_all'])


In [11]:
{
    "question": "...",
    "context": "...",
    "answer": "..."
}

{'question': '...', 'context': '...', 'answer': '...'}

In [16]:
clean_data = []

for sample in train_data:

    question = sample["qa"]["question"]
    answer = sample["qa"]["answer"]

    # Combine textual context
    pre_text = " ".join(sample.get("pre_text", []))
    post_text = " ".join(sample.get("post_text", []))
    table_text = str(sample.get("table", ""))

    combined_context = pre_text + " " + table_text + " " + post_text

    clean_data.append({
        "question": question,
        "context": combined_context,
        "answer": answer
    })

print(clean_data[0])

{'question': 'what is the the interest expense in 2009?', 'context': "interest rate to a variable interest rate based on the three-month libor plus 2.05% ( 2.05 % ) ( 2.34% ( 2.34 % ) as of october 31 , 2009 ) . if libor changes by 100 basis points , our annual interest expense would change by $ 3.8 million . foreign currency exposure as more fully described in note 2i . in the notes to consolidated financial statements contained in item 8 of this annual report on form 10-k , we regularly hedge our non-u.s . dollar-based exposures by entering into forward foreign currency exchange contracts . the terms of these contracts are for periods matching the duration of the underlying exposure and generally range from one month to twelve months . currently , our largest foreign currency exposure is the euro , primarily because our european operations have the highest proportion of our local currency denominated expenses . relative to foreign currency exposures existing at october 31 , 2009 and 

In [23]:
!find ConvFinQA -name "*.json"

In [2]:
import torch
from transformers import AutoTokenizer, AutoModel

class EmbeddingEncoder:
    def __init__(self, model_name="intfloat/e5-base-v2", device="cuda"):
        self.device = device
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(device)
        self.model.eval()

    def encode(self, texts, batch_size=8):
        embeddings = []

        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            encoded = self.tokenizer(
                batch,
                padding=True,
                truncation=True,
                return_tensors="pt"
            ).to(self.device)

            with torch.no_grad():
                outputs = self.model(**encoded)
                emb = outputs.last_hidden_state.mean(dim=1)
                emb = torch.nn.functional.normalize(emb, p=2, dim=1)
                embeddings.append(emb.cpu())

        return torch.cat(embeddings, dim=0)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
encoder = EmbeddingEncoder()
torch.save(encoder.model.state_dict(), "/content/drive/MyDrive/hallucination_model.pt")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: intfloat/e5-base-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from typing import List

class EmbeddingEncoder(nn.Module):
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2", device="cuda"):
        super().__init__()
        self.device = device
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(self.device)
        self.model.eval()

    def forward(self, texts: List[str], batch_size=8):
        all_embeddings = []

        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]

            encoded = self.tokenizer(
                batch,
                padding=True,
                truncation=True,
                return_tensors="pt"
            ).to(self.device)

            with torch.no_grad():
                outputs = self.model(**encoded)
                embeddings = outputs.last_hidden_state.mean(dim=1)
                embeddings = F.normalize(embeddings, p=2, dim=1)
                all_embeddings.append(embeddings.cpu())

        return torch.cat(all_embeddings, dim=0)

In [8]:
class DisagreementScorer:

    def compute_features(self, claim_emb, context_emb):
        """
        claim_emb: (m, d)
        context_emb: (n, d)
        """

        claim_emb = claim_emb.to(device)
        context_emb = context_emb.to(device)

        sim_matrix = torch.matmul(claim_emb, context_emb.T)

        max_support, _ = torch.max(sim_matrix, dim=1)
        mean_support = torch.mean(sim_matrix, dim=1)
        var_support = torch.var(sim_matrix, dim=1)

        disagreement = 1 - max_support

        features = torch.stack([
            max_support,
            mean_support,
            var_support,
            disagreement
        ], dim=1)

        return features

In [9]:
class AdaptiveHallucinationClassifier(nn.Module):
    def __init__(self, input_dim=4, hidden_dim=64):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(hidden_dim, hidden_dim//2),
            nn.ReLU(),

            nn.Linear(hidden_dim//2, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)


In [10]:
class HallucinationDetector:
    def __init__(self):
        self.encoder = EmbeddingEncoder()
        self.scorer = DisagreementScorer()
        self.classifier = AdaptiveHallucinationClassifier().to(device)

    def extract_claims(self, answer: str):
        """
        Simple rule-based claim splitting.
        Replace with LLM-based splitter for production.
        """
        claims = answer.split(".")
        claims = [c.strip() for c in claims if len(c.strip()) > 5]
        return claims

    def build_features(self, claims, contexts):
        claim_emb = self.encoder(claims)
        context_emb = self.encoder(contexts)

        claim_features = self.scorer.compute_features(claim_emb, context_emb)

        # Aggregate over claims (mean pooling)
        aggregated = torch.mean(claim_features, dim=0)
        return aggregated

    def predict(self, answer, contexts):
        claims = self.extract_claims(answer)
        features = self.build_features(claims, contexts)
        features = features.unsqueeze(0).to(device)

        with torch.no_grad():
            prob = self.classifier(features)

        return prob.item()

In [11]:
# Simulated training examples

training_data = [
    {
        "answer": "Aspirin reduces stroke risk by 25 percent in adults over 50.",
        "contexts": [
            "Clinical studies show aspirin reduces stroke risk.",
            "A 25 percent reduction was observed in adults over 50."
        ],
        "label": 0  # Not hallucination
    },
    {
        "answer": "Aspirin increases stroke risk by 40 percent in teenagers.",
        "contexts": [
            "Aspirin reduces stroke risk.",
            "No evidence supports increase in teenagers."
        ],
        "label": 1  # Hallucination
    }
]

In [12]:
def train(detector, data, epochs=10, lr=1e-4):
    optimizer = torch.optim.Adam(detector.classifier.parameters(), lr=lr)
    criterion = nn.BCELoss()

    detector.classifier.train()

    for epoch in range(epochs):
        total_loss = 0

        for sample in data:
            claims = detector.extract_claims(sample["answer"])
            features = detector.build_features(claims, sample["contexts"])
            features = features.unsqueeze(0).to(device)

            label = torch.tensor([[sample["label"]]], dtype=torch.float32).to(device)

            pred = detector.classifier(features)
            loss = criterion(pred, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1} Loss: {total_loss:.4f}")

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
detector = HallucinationDetector()
train(detector, training_data)

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Epoch 1 Loss: 1.4057
Epoch 2 Loss: 1.4437
Epoch 3 Loss: 1.3844
Epoch 4 Loss: 1.3942
Epoch 5 Loss: 1.4241
Epoch 6 Loss: 1.4669
Epoch 7 Loss: 1.4089
Epoch 8 Loss: 1.3694
Epoch 9 Loss: 1.4001
Epoch 10 Loss: 1.4453


In [15]:
detector = HallucinationDetector()
train(detector, training_data)

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Epoch 1 Loss: 1.3826
Epoch 2 Loss: 1.4136
Epoch 3 Loss: 1.3703
Epoch 4 Loss: 1.4215
Epoch 5 Loss: 1.3890
Epoch 6 Loss: 1.3889
Epoch 7 Loss: 1.3888
Epoch 8 Loss: 1.4027
Epoch 9 Loss: 1.4017
Epoch 10 Loss: 1.4035


In [16]:
test_answer = "Aspirin increases stroke risk by 50 percent."
test_contexts = [
    "Aspirin reduces stroke risk.",
    "Studies show benefits in heart patients."
]

prob = detector.predict(test_answer, test_contexts)
print("Hallucination Probability:", prob)

Hallucination Probability: 0.4832291603088379
